## Chapter 8: Entity Framework Core (EF Core)

In the previous chapter, you implemented the Repository pattern using an in‑memory list to store data. While this is great for learning and testing, real applications need a persistent database. **Entity Framework Core (EF Core)** is the modern, lightweight, cross‑platform version of Entity Framework, Microsoft’s recommended Object‑Relational Mapper (ORM) for .NET. It allows you to work with a database using .NET objects, eliminating the need for most of the data‑access code you’d otherwise have to write. In this chapter, you’ll learn how to integrate EF Core into your ASP.NET Core application. We’ll cover database‑first vs. code‑first approaches, setting up `DbContext`, creating entity classes, configuring relationships, and using migrations to evolve your database schema. By the end, you’ll be able to replace your in‑memory repository with a real SQL Server (or other database) implementation.

### 8.1 Database‑First vs. Code‑First Approach

EF Core supports two primary development workflows: **Database‑First** and **Code‑First**.

- **Database‑First**: You start with an existing database. EF Core generates entity classes and a `DbContext` based on the database schema. This approach is useful when you have a legacy database or a database designed by a DBA.
- **Code‑First**: You write your entity classes and `DbContext` in code, and EF Core creates (and updates) the database from your code. This is the industry standard for new applications because it keeps the database schema in sync with your domain model and gives you full control over the code.

In this book, we’ll focus on **Code‑First**, as it aligns with modern development practices and integrates seamlessly with source control and continuous integration.

### 8.2 Setting up `DbContext` and Connection Strings

The `DbContext` class is the primary bridge between your C# code and the database. It represents a session with the database, allowing you to query and save data.

#### Installing EF Core Packages

First, you need to install the EF Core packages. Use the NuGet Package Manager or the .NET CLI.

For SQL Server (the most common provider), run:

```bash
dotnet add package Microsoft.EntityFrameworkCore.SqlServer
dotnet add package Microsoft.EntityFrameworkCore.Tools   # For migrations (optional but recommended)
```

If you prefer another database (e.g., PostgreSQL, SQLite), install the corresponding provider package.

#### Creating the DbContext

Create a folder named `Data` in your project. Inside, add a class called `AppDbContext` that derives from `DbContext`.

```csharp
using Microsoft.EntityFrameworkCore;
using MyFirstMvcApp.Models;

namespace MyFirstMvcApp.Data
{
    public class AppDbContext : DbContext
    {
        public AppDbContext(DbContextOptions<AppDbContext> options)
            : base(options)
        {
        }

        public DbSet<Product> Products { get; set; }
        public DbSet<Category> Categories { get; set; }
    }
}
```

- The constructor takes `DbContextOptions<AppDbContext>` which carries configuration like the database provider and connection string.
- `DbSet<T>` properties represent tables. EF Core will create tables named after these properties (e.g., `Products`, `Categories`).

#### Configuring the Connection String

Connection strings are stored in `appsettings.json` (or per environment). Add a connection string to your `appsettings.json`:

```json
{
  "ConnectionStrings": {
    "DefaultConnection": "Server=(localdb)\\mssqllocaldb;Database=MyFirstMvcAppDb;Trusted_Connection=True;MultipleActiveResultSets=true"
  },
  "Logging": { ... },
  "AllowedHosts": "*"
}
```

This example uses LocalDB, a lightweight SQL Server instance that comes with Visual Studio. For production, you’d use a real SQL Server or Azure SQL.

#### Registering DbContext in DI

In `Program.cs`, register `AppDbContext` with the DI container. You need to specify the database provider and connection string.

```csharp
using Microsoft.EntityFrameworkCore;
using MyFirstMvcApp.Data;

var builder = WebApplication.CreateBuilder(args);

// Add services to the container.
builder.Services.AddControllersWithViews();

// Register DbContext
builder.Services.AddDbContext<AppDbContext>(options =>
    options.UseSqlServer(builder.Configuration.GetConnectionString("DefaultConnection")));

var app = builder.Build();
```

- `AddDbContext` registers `AppDbContext` with **scoped** lifetime (the default, and correct for web apps).
- `UseSqlServer` tells EF Core to use SQL Server with the given connection string.

Now your application is ready to use EF Core.

### 8.3 Creating Entity Classes and Relationships

Entity classes are plain C# classes that map to database tables. They can include properties, validation attributes, and navigation properties for relationships.

#### Simple Entity

Our `Product` class can now be enhanced with EF Core‑specific attributes if needed, but often plain classes suffice.

```csharp
using System.ComponentModel.DataAnnotations;
using System.ComponentModel.DataAnnotations.Schema;

namespace MyFirstMvcApp.Models
{
    public class Product
    {
        [Key]
        public int Id { get; set; }

        [Required]
        [StringLength(100)]
        public string Name { get; set; }

        [Required]
        [Column(TypeName = "decimal(18,2)")]
        public decimal Price { get; set; }

        [StringLength(500)]
        public string Description { get; set; }

        [Required]
        [Display(Name = "Category")]
        public int CategoryId { get; set; }

        public bool IsAvailable { get; set; }

        // Navigation property
        [ForeignKey("CategoryId")]
        public virtual Category Category { get; set; }
    }
}
```

- `[Key]` is optional if the property is named `Id` or `[TypeName]Id`; EF Core will automatically recognize it as the primary key.
- `[Column(TypeName = "decimal(18,2)")]` ensures the SQL column type supports currency precision.
- `CategoryId` is a foreign key. The navigation property `Category` lets you access the related category.

#### Related Entity: Category

```csharp
using System.Collections.Generic;
using System.ComponentModel.DataAnnotations;

namespace MyFirstMvcApp.Models
{
    public class Category
    {
        public int Id { get; set; }

        [Required]
        [StringLength(50)]
        public string Name { get; set; }

        public string Description { get; set; }

        // Navigation property: one category has many products
        public virtual ICollection<Product> Products { get; set; }
    }
}
```

#### Relationships

EF Core can infer relationships based on navigation properties and foreign key properties. Here, a `Category` has many `Products` (one‑to‑many). The `Product` class has a `CategoryId` foreign key and a `Category` navigation property. EF Core will create a relationship with cascade delete behavior (depending on configuration).

You can also configure relationships explicitly using **Fluent API** in `OnModelCreating`. Fluent API is more powerful than data annotations for complex scenarios.

```csharp
protected override void OnModelCreating(ModelBuilder modelBuilder)
{
    // Configure one-to-many relationship
    modelBuilder.Entity<Product>()
        .HasOne(p => p.Category)
        .WithMany(c => c.Products)
        .HasForeignKey(p => p.CategoryId)
        .OnDelete(DeleteBehavior.Restrict); // Prevent cascade delete

    // Additional configuration, e.g., indexes
    modelBuilder.Entity<Product>()
        .HasIndex(p => p.Name)
        .HasDatabaseName("Index_ProductName");
}
```

### 8.4 Migrations: Creating and Updating Your Database Schema

**Migrations** are a way to apply changes to your database schema incrementally, in sync with your model changes. They are version‑controlled and can be applied to any environment (development, staging, production).

#### Creating Your First Migration

With the EF Core Tools installed, open a terminal in the project directory and run:

```bash
dotnet ef migrations add InitialCreate
```

This command:
- Compares your current model (entities and `DbContext`) with the existing database (none yet).
- Scaffolds a migration class in the `Migrations` folder with code to create the database schema.
- Also creates a snapshot of your model (`AppDbContextModelSnapshot.cs`) used for future comparisons.

If you're using Visual Studio, you can use the Package Manager Console and run `Add-Migration InitialCreate`.

#### Applying the Migration

To create or update the database to the latest migration, run:

```bash
dotnet ef database update
```

This executes the `Up` method of all pending migrations against the database specified in the connection string. The database is now created with tables matching your entities.

#### Adding Subsequent Migrations

When you change your model (e.g., add a property to `Product`), create a new migration:

```bash
dotnet ef migrations add AddProductImageUrl
```

This generates a migration with the changes. You can review the generated code (in `Migrations/xxxx_AddProductImageUrl.cs`) and then apply it:

```bash
dotnet ef database update
```

#### Rolling Back Migrations

To revert to a previous migration, use:

```bash
dotnet ef database update LastGoodMigrationName
```

Or remove the last migration (if not applied) with:

```bash
dotnet ef migrations remove
```

#### Generating SQL Scripts

In production, you may want to generate SQL scripts to be reviewed and executed by a DBA:

```bash
dotnet ef migrations script -o migrate.sql
```

### 8.5 Seeding Data

You often need initial data in your database, such as default categories or an admin user. EF Core provides several ways to seed data.

#### Using `OnModelCreating` (simple static data)

```csharp
protected override void OnModelCreating(ModelBuilder modelBuilder)
{
    // Seed categories
    modelBuilder.Entity<Category>().HasData(
        new Category { Id = 1, Name = "Electronics", Description = "Electronic gadgets" },
        new Category { Id = 2, Name = "Books", Description = "All kinds of books" },
        new Category { Id = 3, Name = "Clothing", Description = "Apparel and accessories" }
    );
}
```

When you add a migration after seeding, EF Core will generate insert statements. However, this approach has limitations: you must hardcode IDs, and it can become messy for large datasets.

#### Custom Seeding Logic in `Program.cs`

A more flexible method is to run seeding code after the application starts. You can create an extension method that ensures the database is created and seeded.

```csharp
public static class DbInitializer
{
    public static void Initialize(AppDbContext context)
    {
        // Ensure database is created
        context.Database.EnsureCreated();

        // Look for any categories
        if (context.Categories.Any())
            return; // DB has been seeded

        var categories = new Category[]
        {
            new Category { Name = "Electronics", Description = "Electronic gadgets" },
            new Category { Name = "Books", Description = "All kinds of books" },
            new Category { Name = "Clothing", Description = "Apparel and accessories" }
        };

        context.Categories.AddRange(categories);
        context.SaveChanges();
    }
}
```

Then, in `Program.cs`, after building the app, call it within a scope:

```csharp
using (var scope = app.Services.CreateScope())
{
    var context = scope.ServiceProvider.GetRequiredService<AppDbContext>();
    DbInitializer.Initialize(context);
}
```

This approach is more robust and works well with dependency injection.

### 8.6 Implementing the Repository with EF Core

Now we can replace our `InMemoryProductRepository` with a real EF Core‑based implementation. Create a new class `SqlProductRepository` that implements `IProductRepository`.

```csharp
using Microsoft.EntityFrameworkCore;
using MyFirstMvcApp.Models;
using MyFirstMvcApp.Repositories;
using System.Collections.Generic;
using System.Threading.Tasks;

namespace MyFirstMvcApp.Data.Repositories
{
    public class SqlProductRepository : IProductRepository
    {
        private readonly AppDbContext _context;

        public SqlProductRepository(AppDbContext context)
        {
            _context = context;
        }

        public async Task<IEnumerable<Product>> GetAllAsync()
        {
            return await _context.Products
                .Include(p => p.Category) // eager load category
                .ToListAsync();
        }

        public async Task<Product> GetByIdAsync(int id)
        {
            return await _context.Products
                .Include(p => p.Category)
                .FirstOrDefaultAsync(p => p.Id == id);
        }

        public async Task AddAsync(Product product)
        {
            await _context.Products.AddAsync(product);
            await _context.SaveChangesAsync();
        }

        public async Task UpdateAsync(Product product)
        {
            _context.Entry(product).State = EntityState.Modified;
            await _context.SaveChangesAsync();
        }

        public async Task DeleteAsync(int id)
        {
            var product = await _context.Products.FindAsync(id);
            if (product != null)
            {
                _context.Products.Remove(product);
                await _context.SaveChangesAsync();
            }
        }

        public async Task<bool> ExistsAsync(int id)
        {
            return await _context.Products.AnyAsync(p => p.Id == id);
        }
    }
}
```

- We inject `AppDbContext` via constructor.
- `Include` loads related `Category` data (eager loading) to avoid lazy‑loading issues.
- `SaveChangesAsync` persists changes to the database.
- For updates, we set the entity state to `Modified`.

#### Registering the New Repository

In `Program.cs`, replace the in‑memory repository registration with the SQL version:

```csharp
// builder.Services.AddScoped<IProductRepository, InMemoryProductRepository>();
builder.Services.AddScoped<IProductRepository, SqlProductRepository>();
```

All controller code remains unchanged—the beauty of the repository pattern.

### 8.7 Working with Related Data

Our `Product` list and details views currently display only product properties. With the repository now loading categories, we can display category names.

Update `Products/Index.cshtml`:

```html
@model IEnumerable<Product>

<h1>Product List</h1>

<table class="table">
    <thead>
        <tr>
            <th>ID</th>
            <th>Name</th>
            <th>Price</th>
            <th>Category</th>
            <th>Available</th>
            <th></th>
        </tr>
    </thead>
    <tbody>
    @foreach (var product in Model)
    {
        <tr>
            <td>@product.Id</td>
            <td>@product.Name</td>
            <td>@product.Price.ToString("C")</td>
            <td>@product.Category?.Name</td>  <!-- display category name -->
            <td>@(product.IsAvailable ? "Yes" : "No")</td>
            <td>
                <a asp-action="Details" asp-route-id="@product.Id">Details</a> |
                <a asp-action="Edit" asp-route-id="@product.Id">Edit</a> |
                <a asp-action="Delete" asp-route-id="@product.Id">Delete</a>
            </td>
        </tr>
    }
    </tbody>
</table>
```

Similarly, in `Details.cshtml`:

```html
<dt class="col-sm-2">Category</dt>
<dd class="col-sm-10">@Model.Category?.Name</dd>
```

### 8.8 Handling Concurrency

In web applications, multiple users might try to update the same record simultaneously. EF Core supports **optimistic concurrency** using a concurrency token. You add a property (often a row version) to your entity and configure it as a concurrency token.

Add a `byte[]` property named `RowVersion` to your entity:

```csharp
public class Product
{
    // ...
    [Timestamp]
    public byte[] RowVersion { get; set; }
}
```

The `[Timestamp]` attribute (or Fluent API `.IsRowVersion()`) tells EF Core that this property should be used for concurrency checking. When you update an entity, EF Core includes the original `RowVersion` in the `WHERE` clause. If the row has been modified since you retrieved it, no rows will be affected, and EF Core throws a `DbUpdateConcurrencyException`.

You then handle this exception and decide how to resolve the conflict (e.g., notify the user, refresh data, etc.).

### 8.9 Performance Considerations

- **Asynchronous methods**: Always use the `Async` versions of EF Core methods (`ToListAsync`, `SaveChangesAsync`) to avoid blocking threads.
- **Eager loading vs. lazy loading**: By default, EF Core does not support lazy loading unless you install a separate package. We used eager loading (`Include`). Be careful not to over‑include (the N+1 problem). Sometimes you may want explicit loading or projection.
- **No‑tracking queries**: For read‑only scenarios, use `.AsNoTracking()` to improve performance by skipping change tracking.

```csharp
var products = await _context.Products.AsNoTracking().ToListAsync();
```

- **Pagination**: For large datasets, always paginate using `Skip` and `Take`.

```csharp
var pageSize = 10;
var page = 1;
var products = await _context.Products
    .Skip((page - 1) * pageSize)
    .Take(pageSize)
    .ToListAsync();
```

### 8.10 Summary

In this chapter, you’ve taken a major step toward building production‑ready applications by integrating Entity Framework Core:

- You learned the difference between **Database‑First** and **Code‑First**, and why Code‑First is preferred for new projects.
- You set up `DbContext`, configured a connection string, and registered it with DI.
- You created entity classes and defined relationships using navigation properties and Fluent API.
- You used **migrations** to create and evolve your database schema.
- You implemented a **repository** that uses EF Core, keeping your controllers clean and testable.
- You explored related data loading, concurrency, and performance tips.

Now your application has a persistent database. In the next chapter, **"CRUD Operations with EF Core,"** we’ll dive deeper into querying, inserting, updating, and deleting data using EF Core’s powerful LINQ capabilities. You’ll also learn about the Repository and Unit of Work patterns in more depth, and how to structure your data layer for maintainability and testability.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='7. dependency_injection_and_the_repository_pattern.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='9. crud_operations_with_ef_core.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
